In [1]:
exp_name = 'codegen-350M-mon_pass1_humaneval_py'
sf = f'{exp_name}.jsonl'
#remove '-' from exp_name
td = f'{exp_name.replace("-", "")}'
!python convert_to_pre_eval.py --source_file $sf --target_dir $td

Done.



100%|██████████| 161/161 [00:00<?, ?jsonline/s]


In [2]:
import subprocess
eval_cmd = f"podman run --rm --network none -v ./{td}:/{td}:rw multipl-e-eval --dir /{td} --output-dir /{td} --recursive"
subprocess.run(eval_cmd, shell=True)

CompletedProcess(args='podman run --rm --network none -v ./codegen350Mmon_pass1_humaneval_py:/codegen350Mmon_pass1_humaneval_py:rw multipl-e-eval --dir /codegen350Mmon_pass1_humaneval_py --output-dir /codegen350Mmon_pass1_humaneval_py --recursive', returncode=0)

In [ ]:
!podman run --rm --network none -v ./$td:/$td:rw multipl-e-eval --dir /$td --output-dir /$td --recursive

In [12]:
os.chdir('./MultiPL-E/')

In [22]:
import os
#change directory to parent directory
# os.chdir("..")
target_dir = f'source_generations/{td}'
output_file = f'source_generations/{exp_name}_results.json'
# !python pass_k.py $target_dir $output_file

In [23]:
target_dir = [target_dir]

In [27]:
import numpy as np
from pathlib import Path
import itertools
import argparse
from multipl_e.util import gunzip_json, eprint
import json
import pprint


def estimator(n: int, c: int, k: int) -> float:
    """
    Calculates 1 - comb(n - c, k) / comb(n, k).
    """
    if n - c < k:
        return 1.0
    return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))


def for_file(path):
    data = gunzip_json(path)
    if data is None:
      return None   
    n = len(data["results"])
    c = len([True for r in data["results"] if r["status"] == "OK" and r["exit_code"] == 0])
    return {
        "pass@1": estimator(n, c, 1),
        "pass@10": estimator(n, c, 10),
        "pass@100": estimator(n, c, 100),
        "n": n,
        "temperature": data["temperature"] if "temperature" in data else 0.2
    }

def main():
    res_holder= {
        "dataset": None,
        "pass@k": None,
        "estimate": None,
        "num_problems": None,
        "min_completions": None,
        "max_completions": None,
    }
    results_dict = {}
    print("Dataset,Pass@k,Estimate,NumProblems,MinCompletions,MaxCompletions")
    for d in target_dir:
        results = [ for_file(p) for p in itertools.chain(Path(d).glob("*.results.json"), Path(d).glob("*.results.json.gz")) ]
        results = [ r for r in results if r is not None ]
        name = d.split("/")[-1] if d.split("/")[-1] != "" else d.split("/")[-2]
        temperatures = set(r["temperature"] for r in results)
        if len(temperatures) != 1:
            eprint(f"Found multiple temperatures {temperatures} in {d} {results}")
            continue
        temperature = list(temperatures)[0]
        num_problems = len(results)
        min_completions = np.min([r["n"] for r in results])
        max_completions = np.max([r["n"] for r in results])
        if temperature == 0.2:
            pass_1 = np.mean([r["pass@1"] for r in results])
            print(f"{name},1,{pass_1},{num_problems},{min_completions},{max_completions}")
            res_holder["dataset"] = name
            res_holder["pass@k"] = 1
            results_dict["pass@1"] = res_holder
        elif temperature == 0.8:
            pass_10 = np.mean([r["pass@10"] for r in results])
            pass_100 = np.mean([r["pass@100"] for r in results])
            print(f"{name},10,{pass_10},{num_problems},{min_completions},{max_completions}")
            print(f"{name},100,{pass_100},{num_problems},{min_completions},{max_completions}")
            res_holder["dataset"] = name
            res_holder["pass@k"] = 10
            res_holder["estimate"] = float(pass_1)
            res_holder["num_problems"] = int(num_problems)
            res_holder["min_completions"] = int(min_completions)
            res_holder["max_completions"] = int(max_completions)
            results_dict["pass@10"] = res_holder
            res_holder["pass@k"] = 100
            res_holder["estimate"] = float(pass_100)
            results_dict["pass@100"] = res_holder
        else:
            raise ValueError(f"Unexpected temperature: {temperature}")
        
    
    #pretty print results_dict
    pprint.pprint(results_dict)
    
    if output_file is not None:
        with open(output_file, "w") as f:
            json.dump(results_dict, f)
        print(f"Saved results to {output_file}")
    else:    
        with open("results_dict.json", "w") as f:
            json.dump(results_dict, f)
        
        print("Saved results to results_dict.json")
    

if __name__ == "__main__":
    main()


Dataset,Pass@k,Estimate,NumProblems,MinCompletions,MaxCompletions
codegen350Mmon_pass1_humaneval_py,1,0.024844720496894408,161,1,1
Saved results to source_generations/codegen-350M-mon_pass1_humaneval_py_results.json
{'pass@1': {'dataset': 'codegen350Mmon_pass1_humaneval_py',
            'estimate': 0.024844720496894408,
            'max_completions': 1,
            'min_completions': 1,
            'num_problems': 161,
            'pass@k': 1}}
Saved results to source_generations/codegen-350M-mon_pass1_humaneval_py_results.json
